# LangChain Expression Language (LCEL)

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
openai.api_key = os.environ['OPENAI_API_KEY']

In [2]:
#!pip install pydantic==1.10.8

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema import StrOutputParser

## Simple Chain

In [4]:
prompt = ChatPromptTemplate.from_template(
    "tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

In [5]:
chain = prompt | model | output_parser

In [6]:
chain.invoke({"topic": "Captain America"})

'Why did Captain America start a baking business? \nBecause he wanted to make super-soldiers!'

## More complex chain

And Runnable Map to supply user-provided inputs to the prompt.

In [7]:
from langchain.vectorstores import DocArrayInMemorySearch
from langchain_openai import OpenAIEmbeddings

In [8]:
vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey"],
    embedding=OpenAIEmbeddings()
)
retriever = vectorstore.as_retriever()

In [10]:
retriever.invoke("where did harrison work?")

[Document(page_content='harrison worked at kensho'),
 Document(page_content='bears like to eat honey')]

In [12]:
retriever.invoke("what do bears like to eat")

[Document(page_content='bears like to eat honey'),
 Document(page_content='harrison worked at kensho')]

In [13]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)

In [14]:
from langchain_core.runnables import RunnableMap

In [15]:
chain = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
}) | prompt | model | output_parser

In [16]:
chain.invoke({"question": "where did harrison work?"})

'Harrison worked at Kensho.'

In [17]:
inputs = RunnableMap({
    "context": lambda x: retriever.get_relevant_documents(x["question"]),
    "question": lambda x: x["question"]
})

In [18]:
inputs.invoke({"question": "where did harrison work?"})

{'context': [Document(page_content='harrison worked at kensho'),
  Document(page_content='bears like to eat honey')],
 'question': 'where did harrison work?'}

## Bind

and OpenAI Functions

In [19]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    }
  ]

In [20]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}")
    ]
)
model = ChatOpenAI(temperature=0).bind(functions=functions)

In [21]:
runnable = prompt | model

In [22]:
runnable.invoke({"input": "what is the weather in sf"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"airport_code":"SFO"}', 'name': 'weather_search'}}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 64, 'total_tokens': 80}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-57b4fec2-bb8a-4a3f-8fd8-f93810d74202-0')

In [23]:
functions = [
    {
      "name": "weather_search",
      "description": "Search for weather given an airport code",
      "parameters": {
        "type": "object",
        "properties": {
          "airport_code": {
            "type": "string",
            "description": "The airport code to get the weather for"
          },
        },
        "required": ["airport_code"]
      }
    },
    {
      "name": "sports_search",
      "description": "Search for the news of recent sport events",
      "parameters": {
        "type": "object",
        "properties": {
          "team_name": {
            "type": "string",
            "description": "The sports team to search for"
          },
        },
        "required": ["team_name"]
      }
    }
  ]

In [24]:
model = model.bind(functions=functions)

In [25]:
runnable = prompt | model

In [26]:
runnable.invoke({"input": "how did the patriots do yesterday?"})

AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"team_name":"New England Patriots"}', 'name': 'sports_search'}}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 100, 'total_tokens': 118}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'function_call', 'logprobs': None}, id='run-594ac473-f595-46d0-b67a-7bc94494453e-0')

## Fallbacks

We are using an older version of OpenAI purposefully hence it might fail outputting json which is perfect for out Fallbacks demo

In [45]:
from langchain.llms import OpenAI
import json

In [52]:
simple_model = OpenAI(
    temperature=0, 
    max_tokens=1000, 
    model="text-davinci-001"
    # this is a deprecated model and hence will fail
)
simple_chain = simple_model | json.loads

In [53]:
challenge = "write three poems in a json blob, where each poem is a json blob of a title, author, and first line"

In [54]:
# Note: This will give an error saying model_not_found, which is expected
print(simple_chain.invoke(challenge))

NotFoundError: Error code: 404 - {'error': {'message': 'The model `text-davinci-001` has been deprecated, learn more here: https://platform.openai.com/docs/deprecations', 'type': 'invalid_request_error', 'param': None, 'code': 'model_not_found'}}

**Note**: The next line is expected to fail.

In [55]:
model = ChatOpenAI(temperature=0)
chain = model | StrOutputParser() | json.loads

In [56]:
from pprint import pprint


# This was suppose to fail but its working for some reason I don't know why
pprint(chain.invoke(challenge))

{'poem1': {'author': 'Emily Dickinson',
           'firstLine': 'A rose is a rose is a rose',
           'title': 'The Rose'},
 'poem2': {'author': 'Robert Frost',
           'firstLine': 'Two roads diverged in a yellow wood',
           'title': 'The Road Not Taken'},
 'poem3': {'author': 'Emily Dickinson',
           'firstLine': 'Hope is the thing with feathers',
           'title': 'Hope is the Thing with Feathers'}}


In [57]:
final_chain = simple_chain.with_fallbacks([chain])

In [58]:
pprint(final_chain.invoke(challenge))

{'poem1': {'author': 'Emily Dickinson',
           'firstLine': 'A rose by any other name would smell as sweet',
           'title': 'The Rose'},
 'poem2': {'author': 'Robert Frost',
           'firstLine': 'Two roads diverged in a yellow wood',
           'title': 'The Road Not Taken'},
 'poem3': {'author': 'Emily Dickinson',
           'firstLine': 'Hope is the thing with feathers that perches in the '
                        'soul',
           'title': 'Hope is the Thing with Feathers'}}


## Interface

In [59]:
prompt = ChatPromptTemplate.from_template(
    "Tell me a short joke about {topic}"
)
model = ChatOpenAI()
output_parser = StrOutputParser()

chain = prompt | model | output_parser

In [60]:
chain.invoke({"topic": "bears"})

'Why did the bear break up with his girlfriend? \n\nBecause she was unbearable!'

In [61]:
chain.batch([{"topic": "bears"}, {"topic": "frogs"}])

['Why do bears never wear socks? \nBecause they have bear feet!',
 'Why are frogs so happy? Because they eat whatever bugs them!']

In [62]:
for t in chain.stream({"topic": "bears"}):
    print(t)


Why
 did
 the
 bear
 break
 up
 with
 his
 girlfriend
?
 Because
 he
 couldn
't
 bear
 the
 relationship
 any
 longer
!



In [63]:
response = await chain.ainvoke({"topic": "bears"})
response

"Why did the bear break up with his girlfriend? Because he couldn't bear the relationship anymore!"